In [1]:
#Import the required libraries
import numpy as np
import pandas as pd
import math
import random
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Program Constants

r=1.5
a=1
b=25
c=600
CAP_SCS = 200
CAP_FCS = 400

In [3]:
#Network Tuning Parameters

NETWORK_SLACK = 1.5
NETWORK_FACTOR_2020 = 1  #This can be below 1 or above 1 [Should be around 1]
FCS_SWAPS_PER_ITERATION = 15
FCS_SCS_SWAPS_PER_ITERATION = 3
TRY_SCS = False
FCS_SWAP_CYCLES = 25
FCS_SCS_SWAPS_CYCLES = 10
REBUILD_NETWORK_ATTEMPTS = 10
EPOCHS = 10
FCS_SWAP_REDUCTION_PER_EPOCH = 0
FCS_SCS_SWAP_REDUCTION_PER_EPOCH = 0


In [4]:
df_demand = pd.read_csv('Demand_Forecast_2019_2020.csv')

In [5]:
#df_infra_original = pd.read_csv('EV_infrastructure.csv')
df_infra_original = pd.read_csv('Optimum_Infra_2019_Slack_1.25.csv')

In [6]:
OPTIMUM_NETWORK = None 
OPTIMUM_NETWORK_COST = 1000000000

In [7]:
#Function for computing cost_of_infra
def cost_infra_2019(df_infra):
  total_scs = 0
  total_fcs = 0
  for j in range(len(df_infra)):
    scs_j = df_infra['existing_num_SCS'][j] + df_infra['2019_incr_SCS'][j]
    total_scs += scs_j
    fcs_j = df_infra['existing_num_FCS'][j] + df_infra['2019_incr_FCS'][j]
    total_fcs += fcs_j
  total_cost = total_scs + r * total_fcs
  print(total_scs, total_fcs)
  return total_cost


def cost_infra_2020(df_infra):
  total_scs = 0
  total_fcs = 0
  for j in range(len(df_infra)):
    scs_j = df_infra['existing_num_SCS'][j] + df_infra['2019_incr_SCS'][j] + df_infra['2020_incr_SCS'][j]
    total_scs += scs_j
    fcs_j = df_infra['existing_num_FCS'][j] + df_infra['2019_incr_FCS'][j]  + df_infra['2020_incr_FCS'][j]
    total_fcs += fcs_j
  total_cost = total_scs + r * total_fcs
  print(total_scs, total_fcs)
  return total_cost
    


In [8]:
#Provides the total supply avaiable at supply point j
def Smax_j_2019 (j, df_infra):
  scs_j = df_infra['existing_num_SCS'][j] + df_infra['2019_incr_SCS'][j]
  fcs_j = df_infra['existing_num_FCS'][j] + df_infra['2019_incr_FCS'][j]

  return scs_j * CAP_SCS + fcs_j * CAP_FCS

def Smax_j_2020 (j, df_infra):
  scs_j = df_infra['existing_num_SCS'][j] + df_infra['2019_incr_SCS'][j] + df_infra['2020_incr_SCS'][j]
  fcs_j = df_infra['existing_num_FCS'][j] + df_infra['2019_incr_FCS'][j] + df_infra['2020_incr_FCS'][j]

  return scs_j * CAP_SCS + fcs_j * CAP_FCS


In [9]:
#Provides demand at point i on a given year
def get_D_i (i, year):
  return df_demand[year][i]


In [10]:
#returns the distance between demand point i and supply point j
def distance (i, j):
  x_i = df_demand['x_coordinate'][i]
  y_i = df_demand['y_coordinate'][i]
  x_j = df_infra_original['x_coordinate'][j]
  y_j = df_infra_original['y_coordinate'][j]

  return math.sqrt( (x_i-x_j) **2 +(y_i - y_j) **2)
  

In [11]:
#Increment slow charging station, return true if increment done, false if not done 
def increment_SCS_2019 (j, df_infra):
  if (df_infra['existing_num_FCS'][j] + df_infra['existing_num_SCS'][j] + 
      df_infra['2019_incr_FCS'][j] + df_infra['2019_incr_SCS'][j]) < df_infra['total_parking_slots'][j]:
      
      df_infra['2019_incr_SCS'][j] = df_infra['2019_incr_SCS'][j] + 1

      return True
    
  return False

def increment_SCS_2020 (j, df_infra):
  if (df_infra['existing_num_FCS'][j] + df_infra['existing_num_SCS'][j] + 
      df_infra['2019_incr_FCS'][j] + df_infra['2019_incr_SCS'][j] +
      df_infra['2020_incr_FCS'][j] + df_infra['2020_incr_SCS'][j]) < df_infra['total_parking_slots'][j]:
      
      df_infra['2020_incr_SCS'][j] = df_infra['2020_incr_SCS'][j] + 1

      return True

  return False

In [12]:
def decrement_SCS_2019 (j, df_infra):
  if df_infra['2019_incr_SCS'][j] > 0:
    df_infra['2019_incr_SCS'][j] = df_infra['2019_incr_SCS'][j] - 1

    return True
    
  return False

def decrement_SCS_2020 (j, df_infra):
  if  df_infra['2020_incr_SCS'][j] > 0:
    df_infra['2020_incr_SCS'][j] = df_infra['2020_incr_SCS'][j] - 1
    return True
    
  return False

In [13]:
#Increment fast charging station, return true if increment done, false if not done 
def increment_FCS_2019 (j, df_infra):
  if (df_infra['existing_num_FCS'][j] + df_infra['existing_num_SCS'][j] + 
      df_infra['2019_incr_FCS'][j] + df_infra['2019_incr_SCS'][j]) < df_infra['total_parking_slots'][j]:
      
      df_infra['2019_incr_FCS'][j] = df_infra['2019_incr_FCS'][j] + 1

      return True
    
  return False

def increment_FCS_2020 (j, df_infra):
  if (df_infra['existing_num_FCS'][j] + df_infra['existing_num_SCS'][j] + 
      df_infra['2019_incr_FCS'][j] + df_infra['2019_incr_SCS'][j] +
      df_infra['2020_incr_FCS'][j] + df_infra['2020_incr_SCS'][j]) < df_infra['total_parking_slots'][j]:
      
      df_infra['2020_incr_FCS'][j] = df_infra['2020_incr_FCS'][j] + 1

      return True

  return False

In [14]:
#Decrement fast charging station, return true if increment done, false if not done 
def decrement_FCS_2019 (j, df_infra):
  if df_infra['2019_incr_FCS'][j] > 0:
    df_infra['2019_incr_FCS'][j] = df_infra['2019_incr_FCS'][j] - 1

    return True
    
  return False

def decrement_FCS_2020 (j, df_infra):
  if  df_infra['2020_incr_FCS'][j] > 0:
    df_infra['2020_incr_FCS'][j] = df_infra['2020_incr_FCS'][j] - 1
    return True
    
  return False

In [15]:
#returns the nearest 100 supply points for a demand point
def nearest_supplies(i):
  nearest_100 = dict()
  nearest_distances = dict()
  for x in range (100):
    nearest_100[x] = 1000
  
  for j in range(len(df_infra_original)):
    distance_i_j = distance (i,j)
    for x in range(100):
      if (nearest_100[x] == 1000):
        nearest_100[x] = j
        nearest_distances[j] = distance_i_j
        break
      else:
        x_j = nearest_100[x]
        distance_i_x = nearest_distances[x_j]
        if(distance_i_j < distance_i_x):
          nearest_100[x] = j
          nearest_distances[j] = distance_i_j

          for y in range (x+1, 100):
            temp = nearest_100[y]
            nearest_100[y] = x_j
            x_j = temp
          break
  return list(nearest_100.values())

In [16]:
def total_demand(year):
  return df_demand[year].sum()

def total_supply_2019(df_infra):
  total_supply = 0
  for j in range(len(df_infra)):
    total_supply = total_supply + Smax_j_2019 (j, df_infra)
  return total_supply

def total_supply_2020(df_infra):
  total_supply = 0
  for j in range(len(df_infra)):
    total_supply = total_supply + Smax_j_2020 (j, df_infra)
  return total_supply

In [17]:
def allocate_fast_chargers_randomly(num_chargers, year, df_infra):
  print('Allocating Fast Chargers Randomly')
  for charger in range(num_chargers):
    while (True):
      random_supply_point = random.randint(0, 99)
      flag = False
      if (year == '2019'):
        flag = increment_FCS_2019(random_supply_point, df_infra)
      elif (year == '2020'):
        flag = increment_FCS_2020(random_supply_point, df_infra)
      if flag:
        break

In [18]:
def redistribute_fast_chargers_randomly(year, df_infra):
  
  for charger in range(FCS_SWAPS_PER_ITERATION):
    while(True):
      random_supply_point = random.randint(0, 99)
      flag = False
      if (year == '2019'):
        flag = decrement_FCS_2019(random_supply_point, df_infra)
      elif (year == '2020'):
        flag = decrement_FCS_2020(random_supply_point, df_infra)
      if flag:
        break

  for charger in range(FCS_SWAPS_PER_ITERATION):
    while (True):
      random_supply_point = random.randint(0, 99)
      flag = False
      if (year == '2019'):
        flag = increment_FCS_2019(random_supply_point, df_infra)
      elif (year == '2020'):
        flag = increment_FCS_2020(random_supply_point, df_infra)
      if flag:
        break

In [19]:
def swap_fast_chargers_with_slow_chargers(year, df_infra):
  
  for charger in range(FCS_SCS_SWAPS_PER_ITERATION):
    while(True):
      random_supply_point = random.randint(0, 99)
      flag = False
      if (year == '2019'):
        flag = decrement_FCS_2019(random_supply_point, df_infra)
      elif (year == '2020'):
        flag = decrement_FCS_2020(random_supply_point, df_infra)
      if flag:
        break
  
  for charger in range(FCS_SCS_SWAPS_PER_ITERATION * 2):
    while (True):
      random_supply_point = random.randint(0, 99)
      flag = False
      if (year == '2019'):
        flag = increment_SCS_2019(random_supply_point, df_infra)
      elif (year == '2020'):
        flag = increment_SCS_2020(random_supply_point, df_infra)
      if flag:
        break


In [20]:
#Populates the df_infra_state dataframe with the total capacity of each point
def populate_infra_state(df_infra, df_infra_state, year):
  for j in range(len(df_infra)):
    supply_point_capacity = 0
    if (year == '2019'):
      supply_point_capacity = Smax_j_2019(j, df_infra)
    elif (year == '2020'):
      supply_point_capacity = Smax_j_2020(j, df_infra)
    df_infra_state.loc[len(df_infra_state.index)] = [j, supply_point_capacity, 0, supply_point_capacity]


In [21]:
def build_greedy_network(df_infra_state, df_solution_network, year):
  '''
  Uses a greedy algorithm to build a network, basically looks at the nearest supply 
  point to connect the given demand point. The input variable df_infra_state 
  takes a populated infra_state with appropriate capacities. The df_solution_network
  is modified by ths function with [year, i, j, Dij, values, and cost]  
  '''

  print('Building the network')
  for i in range(len(df_demand)):
    D_i = get_D_i(i, year)
    nearest_supply_points = nearest_supplies(i)
    for j in nearest_supply_points:
      allocated_capacity = df_infra_state['allocated_capacity'][j]
      balance = df_infra_state['balance'][j]
      if D_i <= balance:
        balance = balance - D_i
        allocated_capacity = allocated_capacity + D_i
        cost = distance(i, j) * D_i
        df_solution_network.loc[len(df_solution_network.index)] = [year, 'DS', i, j, D_i, cost]
        D_i = 0
      else:
        D_i = D_i - balance
        allocated_capacity = allocated_capacity + balance
        cost = distance(i, j) * balance
        df_solution_network.loc[len(df_solution_network.index)] = [year, 'DS', i, j, balance, cost]
        balance = 0
      df_infra_state['allocated_capacity'][j] = allocated_capacity
      df_infra_state['balance'][j] = balance
      if D_i == 0:
        break
    if D_i > 0:
      raise Exception('Network cannot be built properly as insufficient infrastructure capacity')
  print('Network Built')  

In [22]:
def build_solution_infra(df_infra, df_solution_infra):
  for j in range(len(df_infra)):
    value = df_infra['existing_num_SCS'][j] + df_infra['2019_incr_SCS'][j]
    cost = c * value
    df_solution_infra.loc[len(df_solution_infra)] = [2019, 'SCS', None, j, value, cost]

  for j in range(len(df_infra)):
    value = df_infra['existing_num_FCS'][j] + df_infra['2019_incr_FCS'][j]
    cost = c * r* value
    df_solution_infra.loc[len(df_solution_infra)] = [2019, 'FCS', None, j, value, cost]
  
  for j in range(len(df_infra)):
    value = df_infra['existing_num_SCS'][j] + df_infra['2019_incr_SCS'][j] + df_infra['2020_incr_SCS'][j]
    cost = c * value
    df_solution_infra.loc[len(df_solution_infra)] = [2020, 'SCS', None, j, value, cost]
  
  for j in range(len(df_infra)):
    value = df_infra['existing_num_FCS'][j] + df_infra['2019_incr_FCS'][j] + df_infra['2020_incr_FCS'][j]
    cost = c * r *  value
    df_solution_infra.loc[len(df_solution_infra)] = [2020, 'FCS', None, j, value, cost]

def compute_infra_cost(df_solution_infra, year):
  if (year == '2019'):
    return list(df_solution_infra.groupby('year').sum()['cost'])[0]
  
  if (year == '2020'):
    return list(df_solution_infra.groupby('year').sum()['cost'])[1]

  

In [23]:
''' Main script '''

#year = '2019'

#incremental_supply_requirement_2019 = total_demand(year) - total_supply_2019(df_infra_original)
#incremental_fast_chargers_2019 = int((incremental_supply_requirement_2019 / CAP_FCS) * (1+ NETWORK_SLACK)) + 1
#df_infra = df_infra_original.copy()
#allocate_fast_chargers_randomly(incremental_fast_chargers_2019, year, df_infra)

year = '2020'

incremental_supply_requirement_2020 = total_supply_2020(df_infra_original) * (total_demand(year) / total_demand('2019') - 1)
incremental_fast_chargers_2020 = max((int((incremental_supply_requirement_2020 / CAP_FCS) * (NETWORK_FACTOR_2020)) + 1), 25)
df_infra = df_infra_original.copy()
allocate_fast_chargers_randomly(incremental_fast_chargers_2020, year, df_infra)

for x in range(EPOCHS):
  print('====================================EPOCH: ', x+1, '===============================================')

  #After every epoch reduce the number of swaps per iteration
  FCS_SWAPS_PER_ITERATION = FCS_SWAPS_PER_ITERATION - FCS_SWAP_REDUCTION_PER_EPOCH
  FCS_SCS_SWAPS_PER_ITERATION = FCS_SCS_SWAPS_PER_ITERATION - FCS_SCS_SWAP_REDUCTION_PER_EPOCH

  for y in range (FCS_SWAP_CYCLES + 1): #+1 has been added as the 1st swap cycle will be skipped
    
    # Start swaps only after the 1st cycle is completed
    if (y > 0): 
      print('FCS SWAP CYCLE: ', y)
      redistribute_fast_chargers_randomly(year, df_infra)

    df_solution_network = pd.DataFrame(columns = ['year', 'data_type', 'demand_point_index', 'supply_point_index', 
                                                  'value', 'cost'])
    df_solution_infra = pd.DataFrame(columns = ['year', 'data_type', 'demand_point_index', 'supply_point_index', 
                                                'value', 'cost'])
    df_infra_state = pd.DataFrame (columns = ['supply_point_index', 'supply_point_capacity', 
                                              'allocated_capacity', 'balance'])

    populate_infra_state(df_infra, df_infra_state, year)
    build_greedy_network(df_infra_state, df_solution_network, year)
    build_solution_infra(df_infra, df_solution_infra)
    
    infra_cost = compute_infra_cost(df_solution_infra, year)
    print('Cost of Infrastructure: ', infra_cost)
    cust_satisfaction_cost = df_solution_network['cost'].sum()
    print('Cost of Customer Unsatisfaction: ', cust_satisfaction_cost)
    total_cost = infra_cost + cust_satisfaction_cost
    print('Total Cost: ', total_cost)

    if(total_cost) < OPTIMUM_NETWORK_COST:
      print('**********New optimum network found with cost: ', total_cost,'**********')
      OPTIMUM_NETWORK_COST = total_cost
      OPTIMUM_NETWORK = {'Optimum Infra': df_infra, 
                      'Solution Network' : df_solution_network, 
                      'Solution Infra': df_solution_infra}

    df_infra = OPTIMUM_NETWORK['Optimum Infra'].copy()
    

  '''
  IMPORTANT: The code below tries to keep adding more slow chargers as long as it finds
             better solutions with additional slow chargers
  '''
  #df_infra_before_SCS_SWAP = OPTIMUM_NETWORK['Optimum Infra'].copy()
  for y in range (FCS_SCS_SWAPS_CYCLES):
    print('SCS SWAP CYCLE: ', y +1)
    #df_infra = df_infra_before_SCS_SWAP.copy()
    swap_fast_chargers_with_slow_chargers(year, df_infra)

    df_solution_network = pd.DataFrame(columns = ['year', 'data_type', 'demand_point_index', 'supply_point_index', 
                                                  'value', 'cost'])
    df_solution_infra = pd.DataFrame(columns = ['year', 'data_type', 'demand_point_index', 'supply_point_index', 
                                                'value', 'cost'])
    df_infra_state = pd.DataFrame (columns = ['supply_point_index', 'supply_point_capacity', 
                                              'allocated_capacity', 'balance'])

    populate_infra_state(df_infra, df_infra_state, year)
    build_greedy_network(df_infra_state, df_solution_network, year)
    build_solution_infra(df_infra, df_solution_infra)
    
    infra_cost = compute_infra_cost(df_solution_infra, year)
    print('Cost of Infrastructure: ', infra_cost)
    cust_satisfaction_cost = df_solution_network['cost'].sum()
    print('Cost of Customer Unsatisfaction: ', cust_satisfaction_cost)
    total_cost = infra_cost + cust_satisfaction_cost
    print('Total Cost: ', total_cost)

    if(total_cost) < OPTIMUM_NETWORK_COST:
      print('**********New optimum network found with cost: ', total_cost,'**********')
      OPTIMUM_NETWORK_COST = total_cost
      OPTIMUM_NETWORK = {'Optimum Infra': df_infra, 
                      'Solution Network' : df_solution_network, 
                      'Solution Infra': df_solution_infra}

    df_infra = OPTIMUM_NETWORK['Optimum Infra'].copy()


Allocating Fast Chargers Randomly
====================================EPOCH:  1 ===============================================
Building the network
Network Built
Cost of Infrastructure:  1525500.0
Cost of Customer Unsatisfaction:  2312407.015828346
Total Cost:  3837907.015828346
**********New optimum network found with cost:  3837907.015828346 **********
FCS SWAP CYCLE:  1
Building the network
Network Built
Cost of Infrastructure:  1525500.0
Cost of Customer Unsatisfaction:  2319732.133853254
Total Cost:  3845232.133853254
FCS SWAP CYCLE:  2
Building the network
Network Built
Cost of Infrastructure:  1525500.0
Cost of Customer Unsatisfaction:  2328974.549897236
Total Cost:  3854474.549897236
FCS SWAP CYCLE:  3
Building the network
Network Built
Cost of Infrastructure:  1525500.0
Cost of Customer Unsatisfaction:  2307094.924189062
Total Cost:  3832594.924189062
**********New optimum network found with cost:  3832594.924189062 **********
FCS SWAP CYCLE:  4
Building the network
Network B

In [24]:
df_infra = OPTIMUM_NETWORK['Optimum Infra']
df_solution_network = OPTIMUM_NETWORK['Solution Network']
df_solution_infra = OPTIMUM_NETWORK['Solution Infra']

df_infra.to_csv('Optimum_Infra_2020.csv')
df_solution_network.to_csv('Solution Network_2020.csv')
df_solution_infra.to_csv ('Solution Infra_2020.csv')

In [31]:
df_solution_infra.shape

(400, 6)